## 1. 뉴스기사 특정 문자열 추출

In [61]:
import re

In [63]:
text = '''
(서울=뉴스1) 권혁준 기자 = 발목 수술을 받게 돼 사실상 정규시즌 복귀가 어려워진 이정후(25·키움 히어로즈). 이정후의 부상은 비로 인해 젖은 외야 잔디에서 비롯됐다.

이정후는 지난 22일 부산 사직구장에서 열린 2023 신한은행 SOL KBO리그 롯데 자이언츠전 경기 도중 왼쪽 발목 통증을 호소해 교체아웃됐다.

정밀 검사 결과 왼쪽 발목 힘줄을 감싸는 막인 신전지대가 손상됐다는 진단이 나왔고 결국 수술을 받게 됐다. 예상 재활 기간은 3개월이다.


25일 서울 고척 스카이돔에서 열리는 한화 이글스전을 앞두고 취재진과 만난 홍원기 키움 감독은 "이정후와 만나 이야기했다"고 했다.

이정후는 부상 당시 상황에 대해 홍 감독에게 설명했다. 홍 감독은 "누런 잔디가 파란 잔디보다 좀 더 부드럽고 질퍽였는데, 이정후가 타구를 잡기 위해 스타트를 하는 상황에서 발이 잔디에 박혀있다가 밀려나면서 통증을 느꼈다고 하더라"고 전했다.

경기 당일 외에도 최근 계속해서 많은 비가 내린 탓에 경기장이 전체적으로 많은 물기를 머금고 있었고, 이것이 부상으로 연결됐다는 설명이다.

이정후가 부상 당한 다음날인 23일에도 경기장에는 비가 오락가락했다. 6회초 키움의 공격 상황에서는 비로 경기가 멈춘 뒤 그라운드 정비 등의 과정을 거쳐 1시간22분이 지연되기도 했다.

간판타자를 허망하게 잃은 만큼 아쉬움이 클 법도 하지만, 홍 감독은 "그라운드 관리를 탓하기엔 무리가 있을 것 같다"고 확대 해석을 경계했다.

그는 "다른 선수들도 똑같은 여건에서 경기를 했다. 이정후만 유독 불운했다고 보는 게 맞을 것 같다"고 말했다.

다만 비에 대한 좋지 않은 감정만은 어쩔 수 없었다. 키움은 올 시즌 많은 비가 내리는 가운데서도 유독 우천 취소 경기가 적어 체력 세이브의 기회가 적었다.

지난달 29일 광주 원정에서는 4-0으로 앞서다 노게임 선언이 돼 반대로 아쉬움을 삼키기도 했다.

홍 감독은 "올해만큼은 정말 비와는 악연인 것 같다"며 쓴웃음을 지었다.

starburyny@news1.kr

Copyright ⓒ 뉴스1코리아 www.news1.kr 무단복제 및 전재 – 재배포금지
'''

### 1.1 이메일 주소 추출

In [3]:
p = re.compile('[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}', flags=re.IGNORECASE)
p.findall(text)

### 1.2 숫자 추출

In [7]:
p = re.compile('[0-9]+')
p.findall(text)

['1',
 '25',
 '22',
 '2023',
 '3',
 '25',
 '23',
 '6',
 '1',
 '22',
 '29',
 '4',
 '0',
 '1',
 '1',
 '1']

### 1.3 기자 이름 추출

In [65]:
target_word = '기자'
# 정규표현식을 사용하여 "are" 단어 앞에 있는 단어를 추출
pattern = r'\b(\w+)\s+{}\b'.format(target_word)
result = re.findall(pattern, text)
if result:
    word_before_target = result[0]
    print("단어 앞에 있는 단어:", word_before_target)
else:
    print("해당 단어가 문자열에 존재하지 않습니다.")

단어 앞에 있는 단어: 권혁준


## 2. 와인 추천 시스템 만들기

In [16]:
import string

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

from nltk.tokenize import word_tokenize, WordPunctTokenizer, sent_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### 2.1 데이터 확인

In [3]:
df = pd.read_csv('c:/reposit/data/bigleader/winemag-data-130k-v2.csv', index_col=0)
df.shape

(129971, 13)

In [4]:
df.head(3)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm


### 2.2 전처리

+ 필요없는 컬럼 제거
+ 영어 소문자로 변경
+ 불용어 제거

#### 2.2.1 필요없는 컬럼 제거

In [5]:
df = df.head(1000)

df = df[['title', 'description']]

In [6]:
df.isnull().sum().to_frame('nan')

,nan
title,0
description,0


#### 2.2.2 description 컬럼 영어 소문자로 변경

In [9]:
df['description'] = df['description'].str.lower()

In [10]:
df.head(3)

,title,description
0,Nicosia 2013 Vulkà Bianco (Etna),"aromas include tropical fruit, broom, brimston..."
1,Quinta dos Avidagos 2011 Avidagos Red (Douro),"this is ripe and fruity, a wine that is smooth..."
2,Rainstorm 2013 Pinot Gris (Willamette Valley),"tart and snappy, the flavors of lime flesh and..."


#### 2.2.3 불용어 및 특수문자 제거

In [17]:
# 특수문자
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [18]:
'?' in string.punctuation

True

In [21]:
# stopwords
sw_eng = stopwords.words('english')

In [13]:
df['description'][0]

"aromas include tropical fruit, broom, brimstone and dried herb. the palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity."

In [35]:
for i in range(len(df)):
    res = []
    for word in word_tokenize(df.loc[i, 'description']):
        
        if (word not in sw_eng) and (word not in string.punctuation):
            res.append(word)
    # print(res)
    
    df.loc[i, 'description_pre'] = ' '.join(res)

In [36]:
df.head(3)

,title,description,description_pre
0,Nicosia 2013 Vulkà Bianco (Etna),"aromas include tropical fruit, broom, brimston...",aromas include tropical fruit broom brimstone ...
1,Quinta dos Avidagos 2011 Avidagos Red (Douro),"this is ripe and fruity, a wine that is smooth...",ripe fruity wine smooth still structured firm ...
2,Rainstorm 2013 Pinot Gris (Willamette Valley),"tart and snappy, the flavors of lime flesh and...",tart snappy flavors lime flesh rind dominate g...


### 2.2 만들기

In [37]:
# 벡터로
tv1 = TfidfVectorizer()

tv1_m = tv1.fit_transform(df['description_pre'])

tv1_m.shape

(1000, 4000)

In [38]:
cs1 = cosine_similarity(tv1_m, tv1_m)
cs1.shape

(1000, 1000)

In [39]:
# 딕셔너리 만들기
# 키: title, 밸류: index
dict1 = dict(zip(df['title'], df.index))

dict1

{'Nicosia 2013 Vulkà Bianco  (Etna)': 0,
 'Quinta dos Avidagos 2011 Avidagos Red (Douro)': 1,
 'Rainstorm 2013 Pinot Gris (Willamette Valley)': 2,
 'St. Julian 2013 Reserve Late Harvest Riesling (Lake Michigan Shore)': 3,
 "Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley)": 4,
 'Tandem 2011 Ars In Vitro Tempranillo-Merlot (Navarra)': 5,
 'Terre di Giurfo 2013 Belsito Frappato (Vittoria)': 6,
 'Trimbach 2012 Gewurztraminer (Alsace)': 7,
 'Heinz Eifel 2013 Shine Gewürztraminer (Rheinhessen)': 8,
 'Jean-Baptiste Adam 2012 Les Natures Pinot Gris (Alsace)': 9,
 'Kirkland Signature 2011 Mountain Cuvée Cabernet Sauvignon (Napa Valley)': 10,
 'Leon Beyer 2012 Gewurztraminer (Alsace)': 11,
 'Louis M. Martini 2012 Cabernet Sauvignon (Alexander Valley)': 12,
 'Masseria Setteporte 2012 Rosso  (Etna)': 13,
 'Mirassou 2012 Chardonnay (Central Coast)': 14,
 'Richard Böcking 2013 Devon Riesling (Mosel)': 15,
 'Felix Lavaque 2010 Felix Malbec (Cafayate)': 16,
 'Gaucho

In [58]:
def recom(title):
    idx = dict1[title]
    
    sim_scores = cs1[idx]
    
    ss = sorted(list(enumerate(cs1[idx])), key=lambda x: x[1], reverse=True)[1:11]
    print(ss)
    res = [i[0] for i in ss]
    
    return df['title'].iloc[res]


In [43]:
sorted(list(enumerate(cs1[1])), key=lambda x: x[1], reverse=True)[1:11]

[(960, 0.24581758346537494),
 (932, 0.23035087260061896),
 (419, 0.2301208563276251),
 (678, 0.22092098300542032),
 (575, 0.2160994733041077),
 (802, 0.2148726661559066),
 (157, 0.21215113074751404),
 (786, 0.19579329861290634),
 (374, 0.19306164047477972),
 (933, 0.1928403931064771)]

In [53]:
recom('Rainstorm 2013 Pinot Gris (Willamette Valley)')

142         Banyan 2007 Riesling (Santa Lucia Highlands)
916    Coelho 2015 Renovação Estate Vineyards Pinot G...
988                     Lachini 2007 Pinot Gris (Oregon)
585    Stevens 2016 StevensSteel Chardonnay (Yakima V...
711    Dutton Estate 2013 Cohen Vineyard Dutton Ranch...
724    Mont Sec 2015 Mont Sec Vineyards Viognier (Texas)
301    Buried Cane 2009 Whiteline No Oak Chardonnay (...
323    J. & F. Lurton 2006 Gran Araucano Cabernet Sau...
58     Tres Palacios 2011 Reserve Pinot Noir (Maipo V...
607    Quady North 2011 Steel-Ox Viognier (Rogue Valley)
Name: title, dtype: object

In [59]:
recom('Nicosia 2013 Vulkà Bianco  (Etna)')

[(677, 0.21425555671361032), (908, 0.20487655699899246), (917, 0.18466847440297396), (862, 0.17491476716486914), (904, 0.17240692614459363), (114, 0.16846170012750317), (159, 0.16731513548928045), (304, 0.13787229118700584), (725, 0.13503249020726027), (8, 0.12599076551880928)]


677    Baracchi Riccardo 2011 Smeriglio Riserva Syrah...
908                    Cascina Bruciata 2013  Barbaresco
917                       Deltetto 2012 Bussia  (Barolo)
862                         Tornatore 2015 Rosato (Etna)
904                        Boffa 2013 Pajè  (Barbaresco)
114                      Pull 2012 BDX Red (Paso Robles)
159    Castello Romitorio 2011 Filo di Seta  (Brunell...
304    Cascina La Ghersa 2004 Vignassa  (Barbera d'As...
725                 Paolo Manzone 2012 Meriame  (Barolo)
8      Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...
Name: title, dtype: object